In [1]:
import librosa, glob, sys, json, pickle, os, random, linecache, statistics
from tqdm import tqdm
import pandas as pd, numpy as np
from IPython.core.display import display, HTML
from collections import Counter
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# setting up random selections
accents = ['kannada_male_english', 'malayalam_male_english', 'rajasthani_male_english', 'hindi_male_english', 
           'tamil_male_english', 'gujarati_female_english', 'manipuri_female_english', 'assamese_female_english']

# random.seed(42)

def _write(manifests_path, items, file):
    with open(manifests_path+file, 'w') as f:
        for item in items:
            json.dump(item, f)
            f.write('\n')

In [5]:
global_files = []
manifests_path = 'random/'

for accent in accents:
	accent_json = open(accent+'/selection.json', 'r')
	files = accent_json.readlines()
	files = [json.loads(file) for file in files]
	random.shuffle(files)
	global_files.extend(files)
	print(len(global_files))

random.shuffle(global_files)
_write(manifests_path, global_files, 'all_selection.json')

3244
6735
10002
13494
18677
21846
28771
34536


In [6]:
!wc -l */selection.json
!wc -l random/all_selection.json

    5765 assamese_female_english/selection.json
    3169 gujarati_female_english/selection.json
    3492 hindi_male_english/selection.json
    3244 kannada_male_english/selection.json
    3491 malayalam_male_english/selection.json
    6925 manipuri_female_english/selection.json
    3267 rajasthani_male_english/selection.json
    5183 tamil_male_english/selection.json
   34536 total
34536 random/all_selection.json


In [7]:
def random_lines(filename, budget)
    idxs = random.sample(range(35329), budget)
    return [linecache.getline(filename, i) for i in idxs]

SyntaxError: invalid syntax (<ipython-input-7-d568b34125cf>, line 1)

In [8]:
def budget(budget_size):
    return int(4.92*budget_size)
# os.makedirs(output_dir, exist_ok=True)
ground_list = [json.loads(line.strip()) for line in open('random/all_selection.json')]

for budget_size in [100, 200, 250, 300, 400, 500, 600, 700, 800, 1000, 2000, 3000, 4000]:
    output_dir = 'random/{}'.format(budget_size)
    list_total_selection, list_total_count, list_total_duration = [], [], []
    list_accent_sample_count, list_accent_sample_duration = {}, {}
    for accent in accents:
        list_accent_sample_count[accent] = []
        list_accent_sample_duration[accent] = []
    
    for i in [1, 2, 3]:
        run = f'run_{i}'
        run_dir = os.path.join(output_dir, run)
        os.makedirs(run_dir, exist_ok=True)
        # for folder in ['train', 'output', 'plots']:
        #     os.makedirs(os.path.join(run_dir, folder))
        all_indices = list(range(len(ground_list)))
        random.seed(i)
        random.shuffle(all_indices)
        total_duration, index = 0, 0
        while total_duration + ground_list[all_indices[index]]['duration'] <= budget(budget_size):
            total_duration += ground_list[all_indices[index]]['duration']
            index += 1
        list_total_count.append(index)
        list_total_duration.append(total_duration)
        selected_indices = all_indices[:index]
        selected_list = [ground_list[j] for j in selected_indices]

#        train_list = selected_list + query_list
        train_list = selected_list
        
        for accent in accents:
            accent_sample_count, accent_sample_duration = 0, 0
            for item in selected_list:
                if item['audio_filepath'].split('/')[-4] == accent:
                    accent_sample_count += 1
                    accent_sample_duration += item['duration']
            list_accent_sample_count[accent].append(accent_sample_count)
            list_accent_sample_duration[accent].append(accent_sample_duration)
        list_total_selection.append(Counter([item['audio_filepath'].split('/')[-4] for item in selected_list]))
        
#        with open(base_dir + query_dir + f'train/error_model/{budget_size}/seed_{i}/train.json', 'w') as f:
#            for line in train_list:
#                f.write('{}\n'.format(json.dumps(line)))
        with open(f'{run_dir}/train.json', 'w') as f:
            for line in train_list:
                f.write('{}\n'.format(json.dumps(line)))
                
#        plots(dirs, run_dir, ground_features, ground_features_Y, query_features, test_features, selected_indices)
    
    stats = 'total selection : ' + ' '.join(map(str, list_total_count)) + ' -> {0:.2f}\n'.format(statistics.mean(list_total_count))
    stats += 'total selection duration: ' + ' '.join(map(str, list_total_duration)) + ' -> {0:.2f}\n'.format(statistics.mean(list_total_duration))
    for accent in accents:
        stats += '\naccent: {}\n'.format(accent)
        stats += 'accented selection: ' + ' '.join(map(str, list_accent_sample_count[accent])) + ' -> {0:.2f}\n'.format(statistics.mean(list_accent_sample_count[accent]))
        stats += 'accented duration: ' + ' '.join(map(str, list_accent_sample_duration[accent])) + ' -> {0:.2f}\n'.format(statistics.mean(list_accent_sample_duration[accent]))
    stats += '\nall selections: ' + str(list_total_selection)
    
    with open(output_dir + '/stats.txt', 'w') as f:
        f.write(stats)


In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import json, os, torch, statistics, glob, librosa, pickle, torchaudio
from tqdm import tqdm
import numpy as np
import torchaudio.functional as F
import torchaudio.transforms as T

mfcc_transform = T.MFCC(
    sample_rate=22050,
    n_mfcc=39,
    melkwargs={
      'n_fft': 2048,
      'n_mels': 256,
      'hop_length': 512,
      'mel_scale': 'htk',
    }
)
base_dir = './'

In [11]:
def extract_features(file_list, file_dir):
    file_type = file_dir.split('/')[-1].replace('.json', '')
    feature_dir = '/'.join(file_dir.split('/')[:-1])+'/39/'
    os.makedirs(os.path.dirname(feature_dir), exist_ok=True)
    feature_file = feature_dir+file_type+'_39.file'
    with open(feature_file, 'wb') as f:
        for file in tqdm(file_list):
            waveform, sample_rate = torchaudio.load(file['audio_filepath'])
            mfcc_features = mfcc_transform(waveform).mean(2).detach().numpy()
            pickle.dump(mfcc_features, f)
    print("completed", file_dir)

In [13]:
speakers = ['kannada_male_english', 'malayalam_male_english', 'rajasthani_male_english', 'hindi_male_english', 
           'tamil_male_english', 'gujarati_female_english', 'manipuri_female_english', 'assamese_female_english']
file_types = ["test", "selection", "seed"]
for speaker in speakers:
    print(speaker)
    for file_type in file_types:
#         print(file_type)
        file_dir = f'{speaker}/{file_type}.json'
        opened_file = open(file_dir)
        json_list = [json.loads(line.strip()) for line in opened_file]
        extract_features(json_list, file_dir)
    print('\n\n')

kannada_male_english
test


100%|██████████| 1433/1433 [00:35<00:00, 40.40it/s]


completed kannada_male_english/test.json
selection


100%|██████████| 3244/3244 [01:09<00:00, 46.82it/s] 


completed kannada_male_english/selection.json
seed


...

completed assamese_female_english/test.json
selection


100%|██████████| 5765/5765 [05:25<00:00, 17.70it/s]


completed assamese_female_english/selection.json
seed


100%|██████████| 50/50 [00:01<00:00, 44.11it/s]


completed assamese_female_english/seed.json





In [12]:
seed_file_dir = 'assamese_female_english/seed.json'
seed_file = open(seed_file_dir)
seed_list = [json.loads(line.strip()) for line in seed_file]
extract_features(seed_list, seed_file_dir)

100%|██████████| 50/50 [00:17<00:00,  2.90it/s]

completed assamese_female_english/seed.json


In [31]:
def duration(json_file):
    dur_list = [json.loads(line.strip())['duration'] for line in open(json_file)]
    print(4.92*100, "requested duration")
    print(int(4.92*len(dur_list)), "is num_selected x 4.92", f"{len(dur_list)} samples")
    print(sum(dur_list), "selected duration")

In [34]:
duration('kannada_male_english/all/budget_200/target_20/FL2MI/39/budget_100/error_model/run_2/train.json')

492.0 requested duration
373 is num_selected x 4.92 76 samples
492.232002 selected duration


In [35]:
!wc -l kannada_male_english/all/budget_200/target_20/FL2MI/39/budget_100/error_model/run_1/train.json

77 kannada_male_english/all/budget_200/target_20/FL2MI/39/budget_100/error_model/run_1/train.json


In [9]:
def convert_json(file_type, speaker):
    ref = open(f'{speaker}/all.json', 'r')
    files = ref.readlines()
    files = [json.loads(file) for file in files]
    path_dict = [(file["audio_filepath"], file["pseudo_text"]) for file in files]
    path_dict = dict(path_dict)
    json_file = open(f"{speaker}/manifests/{file_type}.json", 'r')
    samples = json_file.readlines()
    new_json_file = open(f"{speaker}/{file_type}.json", 'w')
#     new_json_file = open(f"{speaker}/temp.json", 'w')
    new_json_samples = []
    for sample in samples:
        new_dict = json.loads(sample)
        new_dict['pseudo_text'] = path_dict[new_dict['audio_filepath']]
        json.dump(new_dict, new_json_file)
        new_json_file.write('\n')

In [7]:
speakers = ['kannada_male_english', 'malayalam_male_english', 'rajasthani_male_english', 'hindi_male_english', 
           'tamil_male_english', 'gujarati_female_english', 'manipuri_female_english', 'assamese_female_english']
file_types = ["all", "test", "selection", "seed", "seed_plus_dev", "dev"]

In [10]:
for speaker in speakers:
    for file_type in file_types:
        convert_json(file_type, speaker)